[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bluebottle66/Practical-Machine-Learning-Northwestern-/blob/master/Predict422Week3_Kun_Yang.ipynb)

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
RANDOM_SEED = 1
SET_FIT_INTERCEPT = True

In [0]:
import sklearn.linear_model
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt # for root mean-squared error calculation

**Read and Process data**



1.   Switch to github link to store csv files
2.   Look at data statistics, drop n/a, drop neighborhood
3.   Create a new response variable for Log median value of homes in thousands of 1970 dollars
4.   Create preliminary dataset
5.   Perform Standard Scaler of original data




In [0]:
url="https://raw.githubusercontent.com/bluebottle66/Practical-Machine-Learning-Northwestern-/master/boston.csv"
boston_input = pd.read_csv(url)


In [4]:
print('\nboston DataFrame (first and last five rows):')
print(boston_input.head())
print(boston_input.tail())
print('\nGeneral description of the boston_input DataFrame:')
print(boston_input.info())


boston DataFrame (first and last five rows):
  neighborhood     crim    zn  indus  chas    nox  rooms   age     dis  rad  \
0       Nahant  0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1   
1   Swampscott  0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2   
2   Swanpscott  0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2   
3   Marblehead  0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3   
4   Marblehead  0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3   

   tax  ptratio  lstat    mv  
0  296     15.3   4.98  24.0  
1  242     17.8   9.14  21.6  
2  242     17.8   4.03  34.7  
3  222     18.7   2.94  33.4  
4  222     18.7   5.33  36.2  
    neighborhood     crim   zn  indus  chas    nox  rooms   age     dis  rad  \
501     Winthrop  0.06263  0.0  11.93     0  0.573  6.593  69.1  2.4786    1   
502     Winthrop  0.04527  0.0  11.93     0  0.573  6.120  76.7  2.2875    1   
503     Winthrop  0.06076  0.0  11.93     0  0.573  6

In [5]:
print(boston_input.describe())
print(boston_input.columns)

             crim          zn       indus        chas         nox       rooms  \
count  506.000000  506.000000  506.000000  506.000000  506.000000  506.000000   
mean     3.613524   11.363636   11.136779    0.069170    0.554695    6.284634   
std      8.601545   23.322453    6.860353    0.253994    0.115878    0.702617   
min      0.006320    0.000000    0.460000    0.000000    0.385000    3.561000   
25%      0.082045    0.000000    5.190000    0.000000    0.449000    5.885500   
50%      0.256510    0.000000    9.690000    0.000000    0.538000    6.208500   
75%      3.677082   12.500000   18.100000    0.000000    0.624000    6.623500   
max     88.976200  100.000000   27.740000    1.000000    0.871000    8.780000   

              age         dis         rad         tax     ptratio       lstat  \
count  506.000000  506.000000  506.000000  506.000000  506.000000  506.000000   
mean    68.574901    3.795043    9.549407  408.237154   18.455534   12.653063   
std     28.148861    2.1057

In [6]:
boston = boston_input.drop('neighborhood', 1)

boston.dropna()

#create log transformation for response variable
boston['logMv']=np.log(boston['mv'])

prelim_model_data = np.array([boston.mv,\
boston.logMv,\
boston.crim,\
boston.zn,\
boston.indus,\
boston.chas,\
boston.nox,\
boston.rooms,\
boston.age,\
boston.dis,\
boston.rad,\
boston.tax,\
boston.ptratio,\
boston.lstat]).T

# dimensions of the polynomial model X input and y response
# preliminary data before standardization
print('\nData dimensions:', prelim_model_data.shape)


Data dimensions: (506, 14)


In [7]:
#Data transformation
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
print(scaler.fit(prelim_model_data))

# show standardization constants being employed
print(scaler.mean_)
print(scaler.scale_)

# the model data will be standardized form of preliminary model data
model_data = scaler.fit_transform(prelim_model_data)

# dimensions of the polynomial model X input and y response
# all in standardized units of measure
print('\nDimensions for model_data:', model_data.shape)

StandardScaler(copy=True, with_mean=True, with_std=True)
[2.25288538e+01 3.03455800e+00 3.61352356e+00 1.13636364e+01
 1.11367787e+01 6.91699605e-02 5.54695059e-01 6.28463439e+00
 6.85749012e+01 3.79504269e+00 9.54940711e+00 4.08237154e+02
 1.84555336e+01 1.26530632e+01]
[9.17309810e+00 4.07871084e-01 8.59304135e+00 2.32993957e+01
 6.85357058e+00 2.53742935e-01 1.15763115e-01 7.01922514e-01
 2.81210326e+01 2.10362836e+00 8.69865112e+00 1.68370495e+02
 2.16280519e+00 7.13400164e+00]

Dimensions for model_data: (506, 14)


**Model Build up and Data Analysis:**


*   Look at correlation matrix
*   Build up 4 regression models (linear, ridge, lasso and elastic net) on both response variables (home value, with its log transform)
*   Evaluate with cross-validation



In [11]:
corr=boston.corr()
print(corr[(corr>0.5)|(corr<-0.5)])

             crim        zn     indus  chas       nox     rooms       age  \
crim     1.000000       NaN       NaN   NaN       NaN       NaN       NaN   
zn            NaN  1.000000 -0.533828   NaN -0.516604       NaN -0.569537   
indus         NaN -0.533828  1.000000   NaN  0.763651       NaN  0.644779   
chas          NaN       NaN       NaN   1.0       NaN       NaN       NaN   
nox           NaN -0.516604  0.763651   NaN  1.000000       NaN  0.731470   
rooms         NaN       NaN       NaN   NaN       NaN  1.000000       NaN   
age           NaN -0.569537  0.644779   NaN  0.731470       NaN  1.000000   
dis           NaN  0.664408 -0.708027   NaN -0.769230       NaN -0.747881   
rad      0.625505       NaN  0.595129   NaN  0.611441       NaN       NaN   
tax      0.582764       NaN  0.720760   NaN  0.668023       NaN  0.506456   
ptratio       NaN       NaN       NaN   NaN       NaN       NaN       NaN   
lstat         NaN       NaN  0.603800   NaN  0.590879 -0.613808  0.602339   

Only look at the basic correlation with larger than 0.5 or smaller than -0.5: we can find:

positive correlation: rooms

negative correlation: crim, indus, nox,tax,lstat 

In [0]:
#Create list of regression models and all factors in our regression model

names = ['Linear_Regression', 'Ridge_Regression','Lasso_Regression','Elastic_Net_Regression']
variables=['crim','zn','indus','chas','nox','romms','age','dis','rad','tax','ptratio','lstat']

In [0]:
#Create regressors
regressors = [LinearRegression(fit_intercept = SET_FIT_INTERCEPT),
              Ridge(alpha = 1, solver = 'cholesky',
                fit_intercept = SET_FIT_INTERCEPT,
                normalize = False,
                random_state = RANDOM_SEED),
              Lasso(alpha = 0.1, max_iter=10000, tol=0.01,
                fit_intercept = SET_FIT_INTERCEPT,
                random_state = RANDOM_SEED),
              ElasticNet(alpha = 0.1, l1_ratio = 0.5,
                max_iter=10000, tol=0.01,
                fit_intercept = SET_FIT_INTERCEPT,
                normalize = False,
                random_state = RANDOM_SEED)]

**Note: below I am using log transform of response variable to build up regression models. I actually tried both raw repsonse variable and log transformed response variable for test. The models using log transform has smaller RMSE as prediction error.**

In [0]:
N_FOLDS = 10
N_Variables=12

# set up numpy array for storing results

cv_results = np.zeros((N_FOLDS, len(names)))
rsquare_results = np.zeros((N_FOLDS, len(names)))
intercept_results = np.zeros((N_FOLDS, len(names)))
coef_results = np.zeros((N_FOLDS, N_Variables, len(names)))

In [0]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = N_FOLDS, shuffle=False, random_state = RANDOM_SEED)
# check the splitting process by looking at fold observation counts
index_for_fold = 0 # fold count initialized

In [37]:
for train_index, test_index in kf.split(model_data):
  
  print('\nFold index:', index_for_fold,
  '------------------------------------------')

  X_train = model_data[train_index, 2:model_data.shape[1]]
  X_test = model_data[test_index, 2:model_data.shape[1]]
  y_train = model_data[train_index, 1]
  y_test = model_data[test_index, 1]
  # note, if using raw response variable, we shalll use model_data[train_index, 0] here

  print('\nShape of input data for this fold:',
        '\nData Set: (Observations, Variables)')
  print('X_train:', X_train.shape)
  print('X_test:',X_test.shape)
  print('y_train:', y_train.shape)
  print('y_test:',y_test.shape)

  index_for_method = 0 # initialize

  for name, reg_model in zip(names, regressors):
      print('\nRegression model evaluation for:', name)
      print(' Scikit Learn method:', reg_model)
      reg_model.fit(X_train, y_train) # fit on the train set for this fold

      print('Fitted regression intercept:', reg_model.intercept_)
      print('Fitted regression coefficients:', reg_model.coef_)

      intercept_results[index_for_fold, index_for_method]=reg_model.intercept_
      coef_results[index_for_fold, :, index_for_method]=reg_model.coef_[:]

      # evaluate on the test set for this fold
      y_test_predict = reg_model.predict(X_test)
      
      r2_result = r2_score(y_test, y_test_predict)

      fold_method_result = sqrt(mean_squared_error(y_test, y_test_predict))

      print(reg_model.get_params(deep=True))
      print('Root mean-squared error:', fold_method_result)
      
      rsquare_results[index_for_fold, index_for_method] =r2_result

      cv_results[index_for_fold, index_for_method] = fold_method_result
      index_for_method += 1

  index_for_fold += 1
  


Fold index: 0 ------------------------------------------

Shape of input data for this fold: 
Data Set: (Observations, Variables)
X_train: (455, 12)
X_test: (51, 12)
y_train: (455,)
y_test: (51,)

Regression model evaluation for: Linear_Regression
 Scikit Learn method: LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
Fitted regression intercept: 0.0016135906928882518
Fitted regression coefficients: [-0.22453751  0.08125557  0.03118109  0.06662802 -0.22830667  0.1311516
  0.02560665 -0.28078222  0.27515068 -0.2682453  -0.17921273 -0.55544419]
{'copy_X': True, 'fit_intercept': True, 'n_jobs': 1, 'normalize': False}
Root mean-squared error: 0.2957195904826947

Regression model evaluation for: Ridge_Regression
 Scikit Learn method: Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=1, solver='cholesky', tol=0.001)
Fitted regression intercept: 0.0018266239033659856
Fitted regression coefficients: [-0.22341147  0.07935

In [21]:
cv_results_df = pd.DataFrame(cv_results)
cv_results_df.columns = names
print(cv_results_df.mean())

Linear_Regression         0.496452
Ridge_Regression          0.495693
Lasso_Regression          0.544405
Elastic_Net_Regression    0.518855
dtype: float64


**Note: I tried the same code above on raw response variables, the results  are listed below. As we can see, the predict error is higher than log transfrom version**

Linear_Regression 0.561940

Ridge_Regression 0.560511

Lasso_Regression 0.587381

Elastic_Net_Regression 0.568084


In [38]:
r2_results_df = pd.DataFrame(rsquare_results)
r2_results_df.columns = names
print(r2_results_df.mean())

Linear_Regression         0.374328
Ridge_Regression          0.378451
Lasso_Regression          0.339908
Elastic_Net_Regression    0.404225
dtype: float64


In [22]:
intercept_results_df=pd.DataFrame(intercept_results)
intercept_results_df.columns=names
print(intercept_results_df.mean())

Linear_Regression        -0.002315
Ridge_Regression         -0.002277
Lasso_Regression          0.000457
Elastic_Net_Regression   -0.000102
dtype: float64


In [23]:
coef_results_df=pd.DataFrame(np.mean(coef_results,axis=0))
coef_results_df.columns=names
coef_results_df.index=variables
print(coef_results_df)

         Linear_Regression  Ridge_Regression  Lasso_Regression  \
crim             -0.222404         -0.221061         -0.108613   
zn                0.069219          0.067481          0.000000   
indus             0.033600          0.030327          0.000000   
chas              0.066864          0.067334          0.004148   
nox              -0.235590         -0.232229          0.000000   
romms             0.148476          0.150309          0.148735   
age               0.020334          0.018892          0.000000   
dis              -0.263093         -0.260443          0.000000   
rad               0.279546          0.269918          0.000000   
tax              -0.266042         -0.257148         -0.033661   
ptratio          -0.190945         -0.189873         -0.106008   
lstat            -0.528299         -0.525648         -0.505192   

         Elastic_Net_Regression  
crim                  -0.134154  
zn                     0.000000  
indus                  0.000000  
chas 